**The below snippet of code finds is used to find the data features. The features considered for classification are the offset between consecutive x and y coordinates.**

In [0]:
import numpy as np
import json
drawings = []
ct = 0
name = "triangle"
src = "/content/drive/My Drive/AML_Project/RawData/"+name
dst = "/content/drive/My Drive/AML_Project/"+name
with open(src+".ndjson") as f:
    for line in f:
        ct += 1
        if(ct == 70000):
            break
        j_content = json.loads(line)
        drawing = j_content["drawing"]
        points = []
        points.append((0,0))
        stroke_ct = 0
        for stroke in drawing:
            stroke_ct += 1
            for i in range(0,len(stroke[0])):
                    offset_x = stroke[0][i]-stroke[0][i-1]
                    offset_y = stroke[1][i]-stroke[1][i-1]
                    if(i==len(stroke[0])-1 and stroke_ct==len(drawing)):
                        points.append((offset_x,offset_y))
                    elif(i==len(stroke[0])-1):
                        points.append((offset_x,offset_y))
                    else:
                        points.append((offset_x,offset_y))
        drawings.append(points)
drawings = np.array(drawings)
print(drawings.shape)
np.save(dst+"_points.npy",drawings)

**The features are loaded into numpy arrays**

In [0]:
import numpy as np
data_square = np.load("/content/drive/My Drive/AML_Project/NPY_Data/square_points.npy")
data_line = np.load("/content/drive/My Drive/AML_Project/NPY_Data/line_points.npy")
data_hexagon = np.load("/content/drive/My Drive/AML_Project/NPY_Data/hexagon_points.npy")
data_circle = np.load("/content/drive/My Drive/AML_Project/NPY_Data/circle_points.npy")
data_triangle = np.load("/content/drive/My Drive/AML_Project/NPY_Data/triangle_points.npy")

**The below piece of code represents each draing as a set of 100 points**

In [0]:
data = []
train_X = []
labels = []
for i in data_square:
    labels.append([1,0,0,0,0])
    if(len(i)>=100):
        data= i[0:100]
    elif(len(i)<100):
        temp = i
        for j in range(0,100-len(i)):
            temp.append((0,0))
        data = temp
    train_X.append(data)

for i in data_line:
    labels.append([0,1,0,0,0])
    if(len(i)>=100):
        data= i[0:100]
    elif(len(i)<100):
        temp = i
        for j in range(0,100-len(i)):
            temp.append((0,0))
        data = temp
    train_X.append(data)
    
for i in data_hexagon:
    labels.append([0,0,1,0,0])
    if(len(i)>=100):
        data= i[0:100]
    elif(len(i)<100):
        temp = i
        for j in range(0,100-len(i)):
            temp.append((0,0))
        data = temp
    train_X.append(data)
    
for i in data_circle:
    labels.append([0,0,0,1,0])
    if(len(i)>=100):
        data= i[0:100]
    elif(len(i)<100):
        temp = i
        for j in range(0,100-len(i)):
            temp.append((0,0))
        data = temp
    train_X.append(data)

    
for i in data_triangle:
    labels.append([0,0,0,0,1])
    if(len(i)>=100):
        data= i[0:100]
    elif(len(i)<100):
        temp = i
        for j in range(0,100-len(i)):
            temp.append((0,0))
        data = temp
    train_X.append(data)
    
    
print(np.array(train_X).shape)
np.save("/content/drive/My Drive/AML_Project/NPY_Data/data.npy",train_X)
np.save("/content/drive/My Drive/AML_Project/NPY_Data/labels.npy",labels)


**Loading the data features**

In [0]:
import numpy as np
data_X = np.load("/content/drive/My Drive/AML_Project/NPY_Data/data.npy")
data_Y = np.load("/content/drive/My Drive/AML_Project/NPY_Data/labels.npy")

**Including all the imports**

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
import numpy as np
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


**Splitting the data into train and test**

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y, test_size=0.33)
print(X_train.shape)
print(X_test.shape)

**Building the model**

In [0]:
model = Sequential()
model.add(Conv1D(32, kernel_size=2,
                 activation='relu',
                 input_shape=(100,2)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

**Training the data and saving the model**

In [0]:

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=1000,
          epochs=100,
          verbose=1)
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save("/content/drive/My Drive/AML_Project/CNN/model1.h5")

**Loading the model and its weights**

In [0]:
model.load_weights("/content/drive/My Drive/AML_Project/CNN/model_cnn2.h5")

**Using the model for prediction**

In [0]:
pred = model.predict(X_test,batch_size=2000)

**Converting the predictions and ground truth value from a one hot vector into a label (0,1,2,3,4) to represent the 5 geometrical objects.**

In [0]:
predictions = []
for i in pred:
  label = np.argmax(i)
  if(label==0):
    predictions.append(1)
  elif(label==1):
    predictions.append(2)
  elif(label==2):
    predictions.append(3)
  elif(label==3):
    predictions.append(4)
  else:
    predictions.append(5)

    
actual = []
for i in y_test:
  label = np.argmax(i)
  if(label==0):
    actual.append(1)
  elif(label==1):
    actual.append(2)
  elif(label==2):
    actual.append(3)
  elif(label==3):
    actual.append(4)
  else:
    actual.append(5)
    
    

**Obtaining classification report for the models**

In [0]:
from sklearn.metrics import classification_report

print(classification_report(actual,predictions))

             precision    recall  f1-score   support

          1       0.97      0.96      0.96     23093
          2       0.98      0.97      0.97     23176
          3       0.89      0.92      0.91     23061
          4       0.93      0.94      0.94     23217
          5       0.97      0.96      0.97     22952

avg / total       0.95      0.95      0.95    115499

